In [1]:
import random

NumberOfDays = 30

StrategyData = {
    'S1': {'ExpectedReturn': 0.5,  'RiskLevel': 1, 'MaxDrawdown': 1.0},
    'S2': {'ExpectedReturn': 1.2,  'RiskLevel': 2, 'MaxDrawdown': 2.5},
    'S3': {'ExpectedReturn': 2.5,  'RiskLevel': 3, 'MaxDrawdown': 5.0},
    'S4': {'ExpectedReturn': 5.0,  'RiskLevel': 4, 'MaxDrawdown': 10.0},
    'S5': {'ExpectedReturn': 10.0, 'RiskLevel': 5, 'MaxDrawdown': 15.0}
}

StrategyKeys = list(StrategyData.keys())

PopulationSize = 100
NumberOfGenerations = 100
TournamentGroupSize = 5
ElitismCount = 10
MutationProbability = 0.05
CrossoverProbability = 1.0

PenaltySingleDayExceed15 = 10.0
PenaltyCumulativeExceed30 = 20.0
PenaltyConsecutiveRepeat = 5.0

def GenerateRandomChromosome():
    chrom = []
    for _ in range(NumberOfDays):
        chrom.append(random.choice(StrategyKeys))
    return chrom

def CreateInitialPopulation(popSize):
    return [GenerateRandomChromosome() for _ in range(popSize)]

def ComputeFitness(chromosome):
    totalReturn = 0.0
    totalDrawdown = 0.0
    singleDayOver15 = False

    for strategy in chromosome:
        dailyReturn = StrategyData[strategy]['ExpectedReturn']
        dailyDrawdown = StrategyData[strategy]['MaxDrawdown']

        totalReturn += dailyReturn
        totalDrawdown += dailyDrawdown

        if dailyDrawdown > 15.0:
            singleDayOver15 = True

    cumulativeOver30 = (totalDrawdown > 30.0)

    consecutivePenalty = 0.0
    for day in range(1, NumberOfDays):
        if chromosome[day] == chromosome[day - 1]:
            consecutivePenalty += PenaltyConsecutiveRepeat

    if singleDayOver15:
        totalReturn -= PenaltySingleDayExceed15

    if cumulativeOver30:
        totalReturn -= PenaltyCumulativeExceed30

    totalReturn -= consecutivePenalty

    return totalReturn

def TournamentSelection(scoredPop, k):
    sampleGroup = random.sample(scoredPop, k)
    sampleGroup.sort(key=lambda x: x[1], reverse=True)
    return sampleGroup[0][0], sampleGroup[1][0]

def SinglePointCrossover(parentA, parentB):
    if random.random() > CrossoverProbability:
        return parentA[:], parentB[:]

    cut = random.randint(1, NumberOfDays - 1)
    childA = parentA[:cut] + parentB[cut:]
    childB = parentB[:cut] + parentA[cut:]
    return childA, childB

def Mutate(chromosome):
    for i in range(NumberOfDays):
        if random.random() < MutationProbability:
            currentStrat = chromosome[i]
            newStrat = random.choice(StrategyKeys)
            while newStrat == currentStrat:
                newStrat = random.choice(StrategyKeys)
            chromosome[i] = newStrat

def RunGeneticAlgorithm():
    population = CreateInitialPopulation(PopulationSize)

    scoredPopulation = [(individual, ComputeFitness(individual)) for individual in population]

    bestChrom = None
    bestFit = float('-inf')

    for generation in range(NumberOfGenerations):
        scoredPopulation.sort(key=lambda x: x[1], reverse=True)

        if scoredPopulation[0][1] > bestFit:
            bestChrom = scoredPopulation[0][0]
            bestFit = scoredPopulation[0][1]

        nextGen = []
        for i in range(ElitismCount):
            nextGen.append(scoredPopulation[i][0])

        while len(nextGen) < PopulationSize:
            p1, p2 = TournamentSelection(scoredPopulation, TournamentGroupSize)
            c1, c2 = SinglePointCrossover(p1, p2)
            Mutate(c1)
            Mutate(c2)
            nextGen.append(c1)
            if len(nextGen) < PopulationSize:
                nextGen.append(c2)

        scoredPopulation = [(ind, ComputeFitness(ind)) for ind in nextGen]

    scoredPopulation.sort(key=lambda x: x[1], reverse=True)
    return scoredPopulation[0]

# ----------------------_--------------------------- #
bestIndividual, bestFitnessVal = RunGeneticAlgorithm()

print(f"Best Fitness: {bestFitnessVal:.2f}")
print("Best 30-Day Strategy Sequence:")
print(bestIndividual)

totalUnpenalizedReturn = 0.0
totalDrawdownSum = 0.0
anyDayOver15 = False

for dayStrat in bestIndividual:
    totalUnpenalizedReturn += StrategyData[dayStrat]['ExpectedReturn']
    drawdownAmt = StrategyData[dayStrat]['MaxDrawdown']
    totalDrawdownSum += drawdownAmt
    if drawdownAmt > 15.0:
        anyDayOver15 = True

print(f"\nUnpenalized total return over {NumberOfDays} days = {totalUnpenalizedReturn:.2f}%")
print(f"Any single day drawdown exceed 15%? {anyDayOver15}")
print(f"Cumulative drawdown exceed 30%? {totalDrawdownSum > 30.0}")

Best Fitness: 200.00
Best 30-Day Strategy Sequence:
['S5', 'S5', 'S4', 'S5', 'S4', 'S5', 'S4', 'S5', 'S4', 'S5', 'S4', 'S5', 'S4', 'S5', 'S4', 'S5', 'S5', 'S4', 'S5', 'S4', 'S5', 'S4', 'S5', 'S4', 'S5', 'S4', 'S5', 'S4', 'S5', 'S5']

Unpenalized total return over 30 days = 235.00%
Any single day drawdown exceed 15%? False
Cumulative drawdown exceed 30%? True
